In [1]:
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import time
import csv
import re

options = webdriver.ChromeOptions()
options.add_argument('headless')
target_url = 'https://www.tripadvisor.com/Restaurants-g53449-Pittsburgh_Pennsylvania.html'

driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver', chrome_options=options, keep_alive=True)
driver.get(target_url)
driver.maximize_window()

soup = BeautifulSoup(driver.page_source, 'html.parser')
domain = 'https://www.tripadvisor.com'



next_page = "nav next rndBtn ui_button primary taLnk"
page_down = "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;"
check_last_page = soup.findAll("a", {"class": "pageNum taLnk"})
page_list = range(int(soup.findAll("a", {"class": "pageNum taLnk"})[-1].get('data-page-number')))
print("Total number of page: {}".format(len(page_list)))

with open('./data/url_pitt.csv', 'a') as csvfile:
    fieldnames = ['business_id', 'business_name', 'n_comment', 'url']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    index = 0
    
    for p in page_list:
        print('the number of page = {0}/{1}'.format(p+1, len(page_list)))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        bus_blocks = soup.find_all('div', {"class": "restaurants-list-ListCell__cellContainer--2mpJS"})
        
        for bus in bus_blocks:
            index += 1
            bus_name = bus.findAll('a', {"target": "_blank"})[1].text
        
            url = 'https://www.tripadvisor.com/' + bus.findAll('a', {"target": "_blank"})[0].get('href')
            n_comment = bus.findAll('a', {"target": "_blank"})[2].text
            n_comment = re.sub('[^0-9,]', "", n_comment).replace(',','')
            
            writer.writerow(
                            {
                                'business_id':index,
                                'business_name':bus_name,
                                'n_comment':n_comment,
                                #'rank_in_country':rank_in_country,
                                'url':url
                            }
                           )
        try:
            driver.execute_script(page_down)
            time.sleep(5)
            driver.find_element_by_link_text("Next").click()
            time.sleep(8)
        except:
            print('in the end')


In [4]:
crawled = pd.read_csv('./data/url_pitt.csv')
crawled.shape

(870, 4)

In [5]:
crawled['url'].nunique()

870

In [6]:
crawled.drop_duplicates(subset ="url", 
                     keep = 'first', inplace = True) 
crawled.shape

(870, 4)

In [7]:
crawled.head()

,business_id,business_name,n_comment,url
0,1,1. Point Brugge Cafe,408,https://www.tripadvisor.com//Restaurant_Review...
1,2,2. The Melting Pot,416,https://www.tripadvisor.com//Restaurant_Review...
2,3,3. Altius,775,https://www.tripadvisor.com//Restaurant_Review...
3,4,4. Monterey Bay Fish Grotto,987,https://www.tripadvisor.com//Restaurant_Review...
4,5,5. Pamela's Diner,941,https://www.tripadvisor.com//Restaurant_Review...


In [9]:
crawled['n_comment'] = crawled['n_comment'].astype(int)
crawled.sort_values("n_comment", inplace = True, ascending=False)
crawled.head()

,business_id,business_name,n_comment,url
50,51,51. Primanti Bros. Restaurant and Bar,3228,https://www.tripadvisor.com//Restaurant_Review...
45,46,46. Church Brew Works,1871,https://www.tripadvisor.com//Restaurant_Review...
47,48,48. Hofbrauhaus Pittsburgh,1182,https://www.tripadvisor.com//Restaurant_Review...
51,52,52. Meat & Potatoes,1088,https://www.tripadvisor.com//Restaurant_Review...
39,40,40. Grand Concourse,1077,https://www.tripadvisor.com//Restaurant_Review...


In [10]:
new_df = crawled['business_name'].str.split('.', expand=True)
new_df = new_df.loc[:, 0:1]
new_df.columns=['ranking', 'name']
new_df.head()

,ranking,name
50,51,Primanti Bros
45,46,Church Brew Works
47,48,Hofbrauhaus Pittsburgh
51,52,Meat & Potatoes
39,40,Grand Concourse


In [11]:
new_df.shape

(870, 2)

In [12]:
crawled.shape

(870, 4)

In [13]:
final_df = crawled.merge(new_df, left_index=True, right_index=True)
final_df.head()

,business_id,business_name,n_comment,url,ranking,name
50,51,51. Primanti Bros. Restaurant and Bar,3228,https://www.tripadvisor.com//Restaurant_Review...,51,Primanti Bros
45,46,46. Church Brew Works,1871,https://www.tripadvisor.com//Restaurant_Review...,46,Church Brew Works
47,48,48. Hofbrauhaus Pittsburgh,1182,https://www.tripadvisor.com//Restaurant_Review...,48,Hofbrauhaus Pittsburgh
51,52,52. Meat & Potatoes,1088,https://www.tripadvisor.com//Restaurant_Review...,52,Meat & Potatoes
39,40,40. Grand Concourse,1077,https://www.tripadvisor.com//Restaurant_Review...,40,Grand Concourse


In [14]:
final_df.reset_index(inplace=True)
final_df = final_df[['ranking', 'name', 'n_comment', 'url']]
final_df.set_index('ranking', inplace=True)
final_df.head()

,name,n_comment,url
ranking,,,
51,Primanti Bros,3228,https://www.tripadvisor.com//Restaurant_Review...
46,Church Brew Works,1871,https://www.tripadvisor.com//Restaurant_Review...
48,Hofbrauhaus Pittsburgh,1182,https://www.tripadvisor.com//Restaurant_Review...
52,Meat & Potatoes,1088,https://www.tripadvisor.com//Restaurant_Review...
40,Grand Concourse,1077,https://www.tripadvisor.com//Restaurant_Review...


In [15]:
final_df.shape

(870, 3)

In [16]:
final_df.isnull().any()

name         False
n_comment    False
url          False
dtype: bool

In [17]:
final_df.dropna(inplace=True)
final_df.shape

(870, 3)

In [18]:
final_df['name'].str.contains('CLOSED').sum()

0

In [19]:
names = final_df['name'].str.lower()
names.str.contains('closed').sum()

0

In [20]:
final_df.to_csv('./data/pitt_url.csv')

In [21]:
yelp_data = pd.read_csv('./data/selected_bus.csv')
yelp_data.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,fNMVV_ZX7CJSDWQGdOM8Nw,"""Showmars Government Center""",Uptown,"""600 E 4th St""",Charlotte,NC,28202.0,35.221647,-80.839345,3.5,7,1,Restaurants;American (Traditional)
1,Pd52CjgyEU3Rb8co6QfTPw,"""Flight Deck Bar & Grill""",Southeast,"""6730 S Las Vegas Blvd""",Las Vegas,NV,89119.0,36.066914,-115.170848,4.0,13,1,Nightlife;Bars;Barbeque;Sports Bars;American (...
2,4srfPk1s8nlm1YusyDUbjg,"""Subway""",Southeast,"""6889 S Eastern Ave, Ste 101""",Las Vegas,NV,89119.0,36.064652,-115.118954,2.5,6,1,Fast Food;Restaurants;Sandwiches
3,n7V4cD-KqqE3OXk0irJTyA,"""GameWorks""",Southeast,"""6587 Las Vegas Blvd S, Ste 171""",Las Vegas,NV,89119.0,36.068259,-115.178877,3.0,349,1,Arcades;Arts & Entertainment;Gastropubs;Restau...
4,EJFdWX908N8Yc2XG0Lky8A,"""River Moon Cafe""",Lawrenceville,"""104 43rd St""",Pittsburgh,PA,15201.0,40.472735,-79.963265,4.0,5,0,Cafes;Restaurants


In [23]:
yelp_data_pitt = yelp_data[yelp_data['city'] == 'Pittsburgh']
yelp_data_pitt.shape

(2192, 13)

In [24]:
yelp_data_pitt.name = yelp_data_pitt.name.str.replace('"', '')

/Users/jtao/Envs/myenv/lib/python3.4/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [25]:
yelp_data_pitt.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
4,EJFdWX908N8Yc2XG0Lky8A,River Moon Cafe,Lawrenceville,"""104 43rd St""",Pittsburgh,PA,15201.0,40.472735,-79.963265,4.0,5,0,Cafes;Restaurants
5,o1fTwfqN0sDFNpV1CkOPPg,Crazy Mocha Coffee,Lawrenceville,"""4032 Butler St""",Pittsburgh,PA,15201.0,40.469295,-79.961273,3.5,16,1,Restaurants;Sandwiches;Coffee & Tea;Food
6,-nHkhiuerqmfBG3v2v9O-g,Bruegger's Bagels,NaN,"""5217 Clairton Blvd""",Pittsburgh,PA,15236.0,40.344194,-79.970963,2.5,5,1,Breakfast & Brunch;Bagels;Restaurants;Bakeries...
16,mLP-jIIRdoaYM7fJqqFurw,Napoli Pizzeria,Squirrel Hill,"""2006 Murray Ave""",Pittsburgh,PA,15217.0,40.434399,-79.922983,4.0,53,1,Italian;Pizza;Restaurants;Salad
19,j37Z4LIXTH9j6KOq9aX8DQ,Beer Emporium,East Liberty,"""302 S Saint Clair St""",Pittsburgh,PA,15206.0,40.459956,-79.930726,3.5,169,1,Beer;Wine & Spirits;American (New);Restaurants...


In [26]:
yelp_data_pitt.name.values

array(['River Moon Cafe', 'Crazy Mocha Coffee', "Bruegger's Bagels", ...,
       'Cocina Mendoza', 'Brown Bag Delis', 'Tonic Bar & Grill'], dtype=object)

In [27]:
final_df.name = final_df.name.str.strip()

In [28]:
final_df.name.values

array(['Primanti Bros', 'Church Brew Works', 'Hofbrauhaus Pittsburgh',
       'Meat & Potatoes', 'Grand Concourse', "Jerome Bettis' Grille 36",
       'Monterey Bay Fish Grotto', "Pamela's Diner", "Lidia's", 'Eleven',
       'Altius', 'The Capital Grille', "Fat Head's South Side Saloon",
       'Proper Brick Oven and Tap Room', 'Central Diner & Grille',
       'Primanti Bros', 'Nola on the Square', "Pamela's", 'The Commoner',
       'Sienna Mercato', 'Hard Rock Cafe', "DeLuca's Diner", 'Kaya',
       'The Porch at Schenley', "Roland's Seafood Grill",
       'Butcher and the Rye', 'The Melting Pot', 'The Cheesecake Factory',
       'Point Brugge Cafe', "McCormick & Schmick's Seafood & Steaks",
       'Mallorca Restaurant', 'Penn Brewery', "Ditka's", 'Casbah',
       'Ten Penny', 'Le Mont', 'Hyde Park Prime Steakhouse', 'Burgatory',
       "Tessaro's", "Nicky's Thai Kitchen", 'Pizzaiolo Primo', 'Burgatory',
       'Buca di Beppo Italian Restaurant', "Braddock's American Brasserie",
     

In [29]:
# Series(list(set(s1) & set(s2)))
import numpy as np
# np.intersect1d(s1.values,s2.values)
common = list(np.intersect1d(yelp_data_pitt.name.values, final_df.name.values))
len(common)

416

In [30]:
common

['51 Wings and Things',
 "Aiello's Pizza",
 "Aladdin's Eatery",
 "Alida's Woodfire Cucina",
 "Alihan's Mediterranean Cuisine",
 'Allegheny Sandwich Shoppe',
 'Altius',
 'Amazing Wok',
 "Amel's Restaurant",
 'Andora',
 "Andrew's Steak and Seafood",
 "Andy's Sushi Bar",
 'Andys Wine Bar',
 "Angelo's Pizzeria",
 'Angkor Restaurant',
 "Anthony's Coal Fired Pizza",
 "Antonio's Pizzeria",
 "Applebee's",
 'Apsara Cafe',
 'Apteka',
 "Archie's",
 "Ardolino's Pizza",
 "Armstrong's",
 "Arnold's Tea",
 'Arpino Trattoria',
 'Asian Cafe',
 'Asiatique Thai Bistro',
 "Atria's Restaurant",
 'Au Bon Pain',
 "BJ's Restaurant & Brewhouse",
 'BRGR',
 "Bae Bae's Kitchen",
 'Bahama Breeze',
 'Baja Bar & Grill',
 'Bakersfield',
 'Bangal Kabab House and Restaurant',
 'Bangkok Balcony',
 'Banh Mi & Ti',
 'Bar Louie',
 'Bar Marco',
 'Bea Taco Town',
 'Bella Notte',
 "Bellisario's Pizza Palace",
 'Benihana',
 "Benny Fierro's",
 'Biergarten',
 "Big Jim's Restaurant & Bar",
 "Big Shot Bob's House of Wings - Beechvi

In [31]:
selected = yelp_data_pitt[yelp_data_pitt.name.isin(common)][['name', 'is_open']]
selected.shape

(623, 2)

In [32]:
selected['is_open'].value_counts()

1    585
0     38
Name: is_open, dtype: int64

In [33]:
selected.name.nunique()

416

In [34]:
final_df_new = pd.merge(final_df,selected, left_on='name', right_on='name', how='right')
final_df_new.shape

(988, 4)

In [35]:
final_df_new

,name,n_comment,url,is_open
0,Primanti Bros,3228,https://www.tripadvisor.com//Restaurant_Review...,1
1,Primanti Bros,579,https://www.tripadvisor.com//Restaurant_Review...,1
2,Primanti Bros,222,https://www.tripadvisor.com//Restaurant_Review...,1
3,Primanti Bros,132,https://www.tripadvisor.com//Restaurant_Review...,1
4,Primanti Bros,130,https://www.tripadvisor.com//Restaurant_Review...,1
5,Primanti Bros,85,https://www.tripadvisor.com//Restaurant_Review...,1
6,Primanti Bros,74,https://www.tripadvisor.com//Restaurant_Review...,1
7,Primanti Bros,38,https://www.tripadvisor.com//Restaurant_Review...,1
8,Primanti Bros,19,https://www.tripadvisor.com//Restaurant_Review...,1
9,Primanti Bros,3228,https://www.tripadvisor.com//Restaurant_Review...,0


In [36]:
common_df = final_df[final_df.name.isin(common)]
common_df.shape

(481, 3)

In [37]:
common_df

,name,n_comment,url
ranking,,,
51,Primanti Bros,3228,https://www.tripadvisor.com//Restaurant_Review...
46,Church Brew Works,1871,https://www.tripadvisor.com//Restaurant_Review...
48,Hofbrauhaus Pittsburgh,1182,https://www.tripadvisor.com//Restaurant_Review...
52,Meat & Potatoes,1088,https://www.tripadvisor.com//Restaurant_Review...
40,Grand Concourse,1077,https://www.tripadvisor.com//Restaurant_Review...
106,Jerome Bettis' Grille 36,988,https://www.tripadvisor.com//Restaurant_Review...
4,Monterey Bay Fish Grotto,987,https://www.tripadvisor.com//Restaurant_Review...
96,Lidia's,913,https://www.tripadvisor.com//Restaurant_Review...
10,Eleven,801,https://www.tripadvisor.com//Restaurant_Review...


In [38]:
common_df1 = common_df.drop_duplicates(subset='name', keep= False)
common_df1.shape

(381, 3)

In [39]:
result_df = pd.merge(common_df1, selected, how='inner', on='name')
result_df.shape

(475, 4)

In [40]:
result_df

,name,n_comment,url,is_open
0,Church Brew Works,1871,https://www.tripadvisor.com//Restaurant_Review...,1
1,Hofbrauhaus Pittsburgh,1182,https://www.tripadvisor.com//Restaurant_Review...,1
2,Meat & Potatoes,1088,https://www.tripadvisor.com//Restaurant_Review...,1
3,Grand Concourse,1077,https://www.tripadvisor.com//Restaurant_Review...,1
4,Jerome Bettis' Grille 36,988,https://www.tripadvisor.com//Restaurant_Review...,1
5,Monterey Bay Fish Grotto,987,https://www.tripadvisor.com//Restaurant_Review...,1
6,Eleven,801,https://www.tripadvisor.com//Restaurant_Review...,1
7,Altius,775,https://www.tripadvisor.com//Restaurant_Review...,1
8,The Capital Grille,723,https://www.tripadvisor.com//Restaurant_Review...,1
9,Central Diner & Grille,594,https://www.tripadvisor.com//Restaurant_Review...,1


In [41]:
result_df.name.nunique()

381

In [42]:
result_df.drop_duplicates(subset='name', keep='first', inplace=True)
result_df

,name,n_comment,url,is_open
0,Church Brew Works,1871,https://www.tripadvisor.com//Restaurant_Review...,1
1,Hofbrauhaus Pittsburgh,1182,https://www.tripadvisor.com//Restaurant_Review...,1
2,Meat & Potatoes,1088,https://www.tripadvisor.com//Restaurant_Review...,1
3,Grand Concourse,1077,https://www.tripadvisor.com//Restaurant_Review...,1
4,Jerome Bettis' Grille 36,988,https://www.tripadvisor.com//Restaurant_Review...,1
5,Monterey Bay Fish Grotto,987,https://www.tripadvisor.com//Restaurant_Review...,1
6,Eleven,801,https://www.tripadvisor.com//Restaurant_Review...,1
7,Altius,775,https://www.tripadvisor.com//Restaurant_Review...,1
8,The Capital Grille,723,https://www.tripadvisor.com//Restaurant_Review...,1
9,Central Diner & Grille,594,https://www.tripadvisor.com//Restaurant_Review...,1


In [43]:
sum(result_df.is_open == 0)

15

In [44]:
result_df.to_csv('./data/pitt_result20191124.csv')